In [1]:
using DifferentialEquations
using Plots
using Plots.PlotMeasures

# Nagumo equation

We consider the Nagumo equation :

$$
\partial_t u - D \, \partial^2_{xx} u = k \, u^2 (1 - u)
$$


In [ ]:
function nagumo1d!(du, u, p, t)
  k, d, nx, dx = p
  nx = convert(UInt64, nx)
    
  doverdxdx = d/(dx*dx)
  du[1] = doverdxdx * (-2*u[1] + 2*u[2]) + k*u[1]*u[1]*(1-u[1])
  du[2:nx-1] = doverdxdx * (u[1:nx-2] - 2*u[2:nx-1] + u[3:nx]) + k*(u[2:nx-1].^2).*(1 .- u[2:nx-1])
  du[nx] = doverdxdx * (2*u[nx-1] - 2*u[nx]) + k*u[nx]*u[nx]*(1-u[nx])
end

function diff1d!(du, u, p, t)
  d, nx, dx = p
  nx = convert(UInt64, nx)
    
  doverdxdx = d/(dx*dx)
  du[1] = doverdxdx * (-2*u[1] + 2*u[2]) 
  du[2:nx-1] = doverdxdx * (u[1:nx-2] - 2*u[2:nx-1] + u[3:nx])
  du[nx] = doverdxdx * (2*u[nx-1] - 2*u[nx])
end

function reac(u, p, t)
  k = p
  k*(u^2)*(1 - u)
end

function fcn_nagumo1d_exact(k, d, x, t)
  x0 = -20.0
  v = (1/sqrt(2.))*(sqrt(k*d))
  cst  = -(1/sqrt(2.))*(sqrt(k/d))
  u = exp.(cst*(x .-x0 .- v*t)) ./ (1 .+ exp.(cst*(x .- x0 .- v*t)))
end;

## Radau5

In [ ]:
xmin = -50.0
xmax =  50.0
nx = 1001

tini = 0. 
tend = 50.0      
tspan = (tini, tend)

k = 1.0
d = 1.0

x = collect(range(xmin, xmax, length=nx))
uini = fcn_nagumo1d_exact(k, d, x, tini)

dx = (xmax-xmin)/(nx-1)

p = [k, d, nx, dx]

tol = 1e-6
prob = ODEProblem(nagumo1d!, uini, tspan, p)
sol = solve(prob, RadauIIA5(), abstol=tol, reltol=tol)
println("Number of function evaluations : ", sol.destats.nf)
println("Number of linear solves :        ", sol.destats.nsolve)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

uexa = fcn_nagumo1d_exact(k, d, x, tend)

plot(x, [sol[1], sol[end]], label=["t=0" "t=$tend"], size=(900,300), margin=5mm,
     title="Solution", titlefontsize=12, markershape=:xcross, markersize=2)

## Strang

In [ ]:
function solve_strang(tini, tend, nt, uini, p, tol)
    
    k, d, nx, dx = p
    nx = convert(UInt64, nx)
        
    t = collect(range(tini, tend, length=nt))
    dt = (tend-tini)/(nt-1)
        
    u_split = uini
        
    for ti in t[1:end-1]
        
        tspan = (ti, ti+dt/2)
        for inx in 1:nx
            prob = ODEProblem(reac, u_split[inx], tspan, k)
            sol = solve(prob, RadauIIA5(), abstol=tol, reltol=tol)
            u_split[inx] = sol[end]
        end
        
        tspan = (ti, ti+dt)
        prob = ODEProblem(diff1d!, u_split, tspan, [d, nx, dx])
        sol = solve(prob, ROCK4(), abstol=tol, reltol=tol)
        u_split = sol[end]

        tspan = (ti+dt/2, ti+dt)
        for inx in 1:nx
            prob = ODEProblem(reac, u_split[inx], tspan, k)
            sol = solve(prob, RadauIIA5(), abstol=tol, reltol=tol)
            u_split[inx] = sol[end]
        end

    end
    
    u_split
    
end;

In [ ]:
xmin = -50.0
xmax =  50.0
nx = 1001

tini = 0. 
tend = 50.0      
tspan = (tini, tend)

k = 1.0
d = 1.0

x = collect(range(xmin, xmax, length=nx))
uini = fcn_nagumo1d_exact(k, d, x, tini)

dx = (xmax-xmin)/(nx-1)

p = [k, d, nx, dx]

println("Compute quasi-exact solution")
prob = ODEProblem(nagumo1d!, uini, tspan, p)
sol_qexa = solve(prob, RadauIIA5(), abstol=1e-12, reltol=1e-12)

println("Compute Strang solution")
nt_strang = 101
tol = 1e-5
sol_strang = solve_strang(tini, tend, nt_strang, uini, p, tol)

err_qexa  = abs.(sol_strang - sol_qexa[end])

ps = plot(x, [uini, sol_qexa[end], sol_strang], label=["t=0" "t=$tend (quasi-exact)" "t=$tend"], size=(900,300), margin=5mm,
     title="Solution", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(x, err_qexa, xaxis = ("x"), size=(900,600), leg=false, 
          bottom_margin=10mm, title="Error at time $tend (with respect to the quasi-exact solution)", titlefontsize=12, markershape=:xcross, markersize=2)
plot(ps, pe, layout=(2,1))